In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import load_model
from geopy.distance import distance

In [2]:
def evaluate_model(model_path, dataset_path):
    # Load the dataset
    df = pd.read_csv(dataset_path)

    # Define a function to calculate the distance between two sets of lat/lon coordinates
    def calc_distance(home_lat, home_lon, merch_lat, merch_lon):
        home_coords = (home_lat, home_lon)
        merch_coords = (merch_lat, merch_lon)
        return distance(home_coords, merch_coords).miles

    # Calculate the distance between the home and merchant for each transaction
    df['distance'] = df.apply(lambda x: calc_distance(x['lat'], x['long'], x['merch_lat'], x['merch_long']), axis=1)

    # Convert trans_date_trans_time column to datetime format
    df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

    # Extract hour, day of the week, month, and year from trans_date_trans_time column
    df['hour'] = df['trans_date_trans_time'].dt.hour
    df['day_of_week'] = df['trans_date_trans_time'].dt.dayofweek
    df['month'] = df['trans_date_trans_time'].dt.month
    df['year'] = df['trans_date_trans_time'].dt.year

    # Convert dob column to datetime format
    df['dob'] = pd.to_datetime(df['dob'])

    # Calculate age of person at the time of transaction
    df['age'] = (df['trans_date_trans_time'] - df['dob']).astype('<m8[Y]')

    # Drop unnecessary columns
    df = df.drop(['trans_date_trans_time', 'first', 'last', 'dob', 'lat', 'long', 'merch_lat', 'merch_long', 'street', 'city', 'state', 'zip', 'trans_num'], axis=1)

    # Split data into features and target
    X = df.drop('is_fraud', axis=1)
    y = df['is_fraud']

    # One-hot encoding categorical variables
    X = pd.get_dummies(X, columns=['category', 'gender', 'merchant', 'job'])

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize features by removing the mean and scaling to unit variance
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Load the saved model
    model = load_model(model_path)

    # Make predictions on the test set
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5)

    # Output a classification matrix of the prediction accuracy
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [3]:
evaluate_model('model_path.h5', 'test_data.csv')

C:\Users\denni\AppData\Local\Temp\ipykernel_53416\3525874068.py:24: UserWarning: Parsing '21-06-1978' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'])
C:\Users\denni\AppData\Local\Temp\ipykernel_53416\3525874068.py:24: UserWarning: Parsing '19-01-1962' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'])
C:\Users\denni\AppData\Local\Temp\ipykernel_53416\3525874068.py:24: UserWarning: Parsing '28-03-1986' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'])
C:\Users\denni\AppData\Local\Temp\ipykernel_53416\3525874068.py:24: UserWarning: Parsing '19-06-1961' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['dob'] = pd.to_datetime(df['dob'])
C:\Users\denni\AppData\Local\Temp\ip

ValueError: in user code:

    File "d:\Program\anaconda3\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "d:\Program\anaconda3\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Program\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "d:\Program\anaconda3\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "d:\Program\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\Program\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequentia